---
© 2025 KR-Labs. All rights reserved.  
KR-Labs™ is a trademark of Quipu Research Labs, LLC, a subsidiary of Sudiata Giddasira, Inc.

SPDX-License-Identifier: Apache-2.0
---

# Tutorial 3: Volatility Modeling with GARCH

**Author:** KRL Model Zoo Team  
**Affiliation:** KR-Labs  
**Version:** v1.0  
**Date:** October 25, 2025  
**License:** Apache 2.0  
**Tier:** Open-Source (Tier 1-3)

---

## Tutorial Overview

This tutorial demonstrates **volatility modeling** using GARCH (Generalized Autoregressive Conditional Heteroskedasticity) family models, which are essential for modeling time-varying variance in financial and economic time series.

**Models Covered:**
- GARCH(1,1) - Standard model
- EGARCH - Exponential GARCH (asymmetric effects)
- GJR-GARCH - Leverage effects

**Dataset:** Synthetic financial returns data (daily, 1000 observations)

### Learning Objectives

By the end of this tutorial, you will be able to:
1. Model time-varying volatility in financial returns
2. Detect volatility clustering and leverage effects
3. Compare GARCH, EGARCH, and GJR-GARCH models
4. Generate volatility forecasts for risk management

### Prerequisites

- Understanding of financial time series (returns, volatility)
- Basic knowledge of conditional heteroskedasticity
- Familiarity with risk metrics (VaR, CVaR)

**Estimated Time:** 40-50 minutes

---

## Business Applications

GARCH models are critical for:

- **Risk Management:** Calculate Value at Risk (VaR) and Expected Shortfall
- **Portfolio Optimization:** Model covariance matrices for asset allocation
- **Option Pricing:** Estimate volatility for derivatives pricing
- **Regulatory Compliance:** Basel III capital requirements and stress testing

---

## Data Provenance

**Source:** Synthetic data generated for educational purposes  
**Characteristics:**
- Daily frequency (1000 observations)
- GARCH(1,1) process with parameters: ω=0.00001, α=0.1, β=0.85
- 12.5% annualized drift (0.05% daily)
- Volatility clustering pattern
- Leverage effect (negative returns increase volatility more)

**Real-world Equivalents:**
- Stock market returns (S&P 500, individual stocks)
- Currency exchange rates (FX volatility)
- Commodity prices (oil, gold)

## Setup

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Import KRL Model Zoo
from krl_models.volatility import GARCH, EGARCH, GJRGARCH

# Set random seed for reproducibility
np.random.seed(42)

# Configure plotting
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)

## Load and Explore Data

In [ ]:
# Load financial returns data
data_path = Path('../data/financial_returns_sample.csv')
df = pd.read_csv(data_path)
df['date'] = pd.to_datetime(df['date'])

print("Dataset shape:", df.shape)
print("\nFirst few rows:")
print(df.head())
print("\nData summary:")
print(df.describe())

In [ ]:
# Visualize returns and volatility
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Returns plot
axes[0].plot(df['date'], df['returns'], linewidth=0.8, color='steelblue', alpha=0.7)
axes[0].axhline(y=0, color='red', linestyle='--', linewidth=1)
axes[0].set_title('Daily Returns', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Returns (%)', fontsize=12)
axes[0].grid(True, alpha=0.3)

# Volatility plot
axes[1].plot(df['date'], df['volatility'], linewidth=1.5, color='darkred')
axes[1].set_title('Conditional Volatility', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Date', fontsize=12)
axes[1].set_ylabel('Volatility', fontsize=12)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nReturns statistics:")
print(f"  Mean: {df['returns'].mean():.4f}%")
print(f"  Std Dev: {df['returns'].std():.4f}%")
print(f"  Skewness: {df['returns'].skew():.4f}")
print(f"  Kurtosis: {df['returns'].kurtosis():.4f}")

## Train-Test Split

Split data into training (80%) and testing (20%) sets.

In [ ]:
# Split data: 80% train, 20% test
train_size = int(len(df) * 0.8)
train_data = df[:train_size].copy()
test_data = df[train_size:].copy()

print(f"Training set: {len(train_data)} observations")
print(f"Test set: {len(test_data)} observations")

## Model 1: GARCH(1,1)

In [ ]:
# Initialize and fit GARCH(1,1) model
garch_model = GARCH(p=1, q=1)
result_garch = garch_model.fit(train_data['returns'].values)

print("GARCH(1,1) model fitted successfully!")
print(f"\nModel parameters:")
params = result_garch.payload.get('params', {})
print(f"  ω (omega): {params.get('omega', 'N/A'):.6f}")
print(f"  α (alpha): {params.get('alpha[1]', 'N/A'):.6f}")
print(f"  β (beta):  {params.get('beta[1]', 'N/A'):.6f}")
print(f"\nLog-Likelihood: {result_garch.payload.get('loglikelihood', 'N/A')}")
print(f"AIC: {result_garch.payload.get('aic', 'N/A')}")
print(f"BIC: {result_garch.payload.get('bic', 'N/A')}")

In [ ]:
# Make volatility forecasts
forecast_garch = garch_model.predict(periods=len(test_data))
volatility_forecast_garch = forecast_garch.volatility

# Calculate RMSE
from sklearn.metrics import mean_squared_error
rmse_garch = np.sqrt(mean_squared_error(test_data['volatility'], volatility_forecast_garch))

print(f"GARCH(1,1) Volatility Forecast RMSE: {rmse_garch:.6f}")

## Model 2: EGARCH (Asymmetric Effects)

In [ ]:
# Initialize and fit EGARCH model
egarch_model = EGARCH(p=1, q=1)
result_egarch = egarch_model.fit(train_data['returns'].values)

print("EGARCH(1,1) model fitted successfully!")
print(f"\nModel parameters:")
params_e = result_egarch.payload.get('params', {})
print(f"  ω (omega): {params_e.get('omega', 'N/A'):.6f}")
print(f"  α (alpha): {params_e.get('alpha[1]', 'N/A'):.6f}")
print(f"  γ (gamma): {params_e.get('gamma[1]', 'N/A'):.6f} (asymmetry)")
print(f"  β (beta):  {params_e.get('beta[1]', 'N/A'):.6f}")

# Make forecasts
forecast_egarch = egarch_model.predict(periods=len(test_data))
volatility_forecast_egarch = forecast_egarch.volatility
rmse_egarch = np.sqrt(mean_squared_error(test_data['volatility'], volatility_forecast_egarch))

print(f"\nEGARCH Volatility Forecast RMSE: {rmse_egarch:.6f}")

## Model 3: GJR-GARCH (Leverage Effects)

In [ ]:
# Initialize and fit GJR-GARCH model
gjr_model = GJRGARCH(p=1, q=1)
result_gjr = gjr_model.fit(train_data['returns'].values)

print("GJR-GARCH(1,1) model fitted successfully!")
print(f"\nModel parameters:")
params_gjr = result_gjr.payload.get('params', {})
print(f"  ω (omega): {params_gjr.get('omega', 'N/A'):.6f}")
print(f"  α (alpha): {params_gjr.get('alpha[1]', 'N/A'):.6f}")
print(f"  γ (gamma): {params_gjr.get('gamma[1]', 'N/A'):.6f} (leverage)")
print(f"  β (beta):  {params_gjr.get('beta[1]', 'N/A'):.6f}")

# Make forecasts
forecast_gjr = gjr_model.predict(periods=len(test_data))
volatility_forecast_gjr = forecast_gjr.volatility
rmse_gjr = np.sqrt(mean_squared_error(test_data['volatility'], volatility_forecast_gjr))

print(f"\nGJR-GARCH Volatility Forecast RMSE: {rmse_gjr:.6f}")

## Model Comparison

In [ ]:
# Create comparison dataframe
comparison = pd.DataFrame({
    'Model': ['GARCH(1,1)', 'EGARCH(1,1)', 'GJR-GARCH(1,1)'],
    'RMSE': [rmse_garch, rmse_egarch, rmse_gjr],
    'AIC': [
        result_garch.payload.get('aic', np.nan),
        result_egarch.payload.get('aic', np.nan),
        result_gjr.payload.get('aic', np.nan)
    ],
    'BIC': [
        result_garch.payload.get('bic', np.nan),
        result_egarch.payload.get('bic', np.nan),
        result_gjr.payload.get('bic', np.nan)
    ]
})

comparison = comparison.sort_values('RMSE')
print("\nModel Performance Comparison:")
print(comparison.to_string(index=False))

# Visualize volatility forecasts
plt.figure(figsize=(14, 7))

plt.plot(test_data['date'], test_data['volatility'], 
         label='Actual Volatility', linewidth=2, color='black')
plt.plot(test_data['date'], volatility_forecast_garch, 
         label='GARCH(1,1)', linewidth=1.5, linestyle='--', color='steelblue')
plt.plot(test_data['date'], volatility_forecast_egarch, 
         label='EGARCH(1,1)', linewidth=1.5, linestyle='--', color='coral')
plt.plot(test_data['date'], volatility_forecast_gjr, 
         label='GJR-GARCH(1,1)', linewidth=1.5, linestyle='--', color='green')

plt.title('Volatility Forecasts Comparison', fontsize=16, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Conditional Volatility', fontsize=12)
plt.legend(loc='best', fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Key Takeaways

1. **GARCH Models:**
   - Capture volatility clustering (high volatility followed by high volatility)
   - Model time-varying conditional variance
   - Essential for financial risk management

2. **Model Variants:**
   - **GARCH(1,1):** Standard model, symmetric response to shocks
   - **EGARCH:** Exponential specification, allows asymmetric effects
   - **GJR-GARCH:** Threshold model, captures leverage effect (negative shocks increase volatility more)

3. **Parameter Interpretation:**
   - **ω (omega):** Baseline volatility level
   - **α (alpha):** ARCH effect (short-term persistence)
   - **β (beta):** GARCH effect (long-term persistence)
   - **γ (gamma):** Asymmetry/leverage parameter

4. **Business Applications:**
   - Value at Risk (VaR) calculations
   - Portfolio risk management
   - Option pricing and derivatives
   - Regulatory capital requirements

## Export Results & Reproducibility

This section exports model results, visualizations, and metadata for reproducibility.

In [ ]:
from datetime import datetime
from pathlib import Path
import json

# Create output directory
output_dir = Path('../outputs') / f'volatility_modeling_{datetime.now().strftime("%Y%m%d_%H%M%S")}'
output_dir.mkdir(parents=True, exist_ok=True)

print(f"Exporting results to: {output_dir}\n")

# 1. Export data
train_data.to_csv(output_dir / 'train_data.csv', index=False)
test_data.to_csv(output_dir / 'test_data.csv', index=False)
print(" Exported training and test data")

# 2. Export volatility forecasts
forecast_df = pd.DataFrame({
    'date': test_data['date'],
    'actual_volatility': test_data['volatility'].values,
    'garch_forecast': volatility_forecast_garch,
    'egarch_forecast': volatility_forecast_egarch,
    'gjr_forecast': volatility_forecast_gjr
})
forecast_df.to_csv(output_dir / 'volatility_forecasts.csv', index=False)
print(" Exported volatility forecasts")

# 3. Export model comparison
comparison.to_csv(output_dir / 'model_comparison.csv', index=False)
print(" Exported model comparison")

# 4. Export metadata
metadata = {
    "tutorial": "03_volatility_modeling_garch.ipynb",
    "version": "v1.0",
    "execution_date": datetime.now().isoformat(),
    "models": ["GARCH(1,1)", "EGARCH(1,1)", "GJR-GARCH(1,1)"],
    "dataset": {
        "name": "financial_returns_sample.csv",
        "records": len(df),
        "train_size": len(train_data),
        "test_size": len(test_data)
    },
    "performance": {
        "garch": {"rmse": float(rmse_garch), "aic": float(result_garch.payload.get('aic', np.nan))},
        "egarch": {"rmse": float(rmse_egarch), "aic": float(result_egarch.payload.get('aic', np.nan))},
        "gjr": {"rmse": float(rmse_gjr), "aic": float(result_gjr.payload.get('aic', np.nan))}
    },
    "reproducibility": {
        "random_seed": 42,
        "python_version": "3.9+",
        "required_packages": ["krl-model-zoo", "pandas", "numpy", "matplotlib", "arch"]
    }
}

with open(output_dir / 'execution_metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2)
print(" Exported execution metadata")

print(f"\n{'='*60}")
print("EXPORT COMPLETE")
print(f"{'='*60}")

## Responsible Use & Limitations

### Ethical Considerations

1. **Data Privacy:**
   - This analysis uses synthetic financial data for demonstration
   - Real applications should use publicly available market data
   - Never use insider information or confidential data

2. **Bias & Fairness:**
   - Volatility models reflect historical market behavior
   - Past volatility patterns may not predict future crises
   - Consider market regime changes and structural breaks

3. **Limitations:**
   - Synthetic data for demonstration purposes only
   - Real risk models require extensive validation and backtesting
   - GARCH models assume no structural breaks or regime changes
   - Models may underestimate tail risks and extreme events
   - Volatility forecasts degrade quickly beyond short horizons

4. **Recommended Use Cases:**
   -  Educational purposes and learning
   -  Risk measurement and portfolio management
   -  Scenario analysis and stress testing
   -  Option pricing and derivatives valuation
   -  High-frequency trading without proper risk controls
   -  Regulatory capital calculations without expert validation
   -  Client advisory without comprehensive risk disclosures

5. **Model Assumptions:**
   - Returns follow a stationary process with time-varying variance
   - Volatility persistence parameters are stable over time
   - No structural breaks or regime changes in volatility dynamics
   - Distribution of standardized residuals is correct

### Best Practices

- Always validate models on out-of-sample test data
- Use multiple model specifications and compare performance
- Conduct regular model backtesting and validation
- Monitor model performance during periods of market stress
- Update models frequently with new data
- Implement proper risk management and position limits
- Consult risk management experts for production systems
- Document all assumptions, limitations, and model changes

For questions about responsible use: info@krlabs.dev

## References

1. **Bollerslev, T.** (1986). Generalized autoregressive conditional heteroskedasticity. *Journal of Econometrics*, 31(3), 307-327.

2. **Engle, R. F.** (1982). Autoregressive conditional heteroscedasticity with estimates of the variance of United Kingdom inflation. *Econometrica*, 50(4), 987-1007.

3. **Nelson, D. B.** (1991). Conditional heteroskedasticity in asset returns: A new approach. *Econometrica*, 59(2), 347-370.

4. **Glosten, L. R., Jagannathan, R., & Runkle, D. E.** (1993). On the relation between the expected value and the volatility of the nominal excess return on stocks. *Journal of Finance*, 48(5), 1779-1801.

5. **Hansen, P. R., & Lunde, A.** (2005). A forecast comparison of volatility models: does anything beat a GARCH(1,1)? *Journal of Applied Econometrics*, 20(7), 873-889.

6. **Tsay, R. S.** (2010). *Analysis of Financial Time Series* (3rd ed.). Wiley.

---

## Citation

To cite this tutorial:

```bibtex
@misc{krl_garch_volatility_2025,
  title = {Tutorial 3: Volatility Modeling with GARCH},
  author = {KRL Model Zoo Team},
  year = {2025},
  publisher = {KR-Labs},
  url = {https://github.com/KR-Labs/krl-model-zoo},
  note = {Tutorial from KRL Model Zoo v1.0.0}
}
```

To cite KRL Model Zoo:

```bibtex
@software{krl_model_zoo_2025,
  title = {KRL Model Zoo: Production-Grade Models for Socioeconomic Analysis},
  author = {KR-Labs},
  year = {2025},
  url = {https://github.com/KR-Labs/krl-model-zoo},
  version = {1.0.0},
  license = {Apache-2.0}
}
```

---

<div style="text-align: center; padding: 20px 0; border-top: 2px solid #333; margin-top: 40px;">
  <p style="font-size: 14px; font-weight: bold; margin-bottom: 5px;">
    KR-Labs | Data-Driven Economic Analysis
  </p>
  <p style="font-size: 12px; color: #666; margin: 5px 0;">
    Contact: <a href="mailto:info@krlabs.dev">info@krlabs.dev</a>
  </p>
  <p style="font-size: 11px; color: #666; margin: 5px 0;">
    © 2025 KR-Labs. All rights reserved.<br>
    <strong>KR-Labs™</strong> is a trademark of Quipu Research Labs, LLC, a subsidiary of Sudiata Giddasira, Inc.
  </p>
  <p style="font-size: 11px; color: #666; margin: 5px 0;">
    <a href="https://www.apache.org/licenses/LICENSE-2.0" target="_blank">Apache 2.0 License</a> | 
    <a href="https://github.com/KR-Labs/krl-model-zoo" target="_blank">GitHub Repository</a>
  </p>
</div>